In [26]:
!pip install summa

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for summa: filename=summa-1.2.0-cp36-none-any.whl size=54411 sha256=4cf89af24b8d55db6b0caf7bd14c163cefb4c2e5c63029ad39fe83635a199407
  Stored in directory: /root/.cache/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [9]:
!pip install python-rake
!pip install pymorphy2

In [2]:
!pip install simplejson

     |████████████████████████████████| 133kB 5.9MB/s 


In [130]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
stop = stopwords.words('english')
from summa import keywords
from gensim.summarization import keywords as kw
rake = RAKE.Rake(stop)
import gzip
import simplejson
import pandas as pd
import re

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
def parse(filename, r):
    f = gzip.open(filename, r)
    entry = {}
    for l in f:
        l = l.strip()
        colonPos = l.find(b':')
        if colonPos == -1:
            yield entry
            entry = {}
            continue
        eName = l[:colonPos].decode('utf-8')
        rest = l[colonPos+2:].decode('utf-8')
        entry[eName] = rest
    yield entry

In [128]:
d = []
for e in parse("Musical_Instruments.txt.gz", 'r'):
    d.append(e)
data = pd.DataFrame.from_dict(d)

In [133]:
# беру не все данные, потому что долго работает код
data = data[:10000]

1) Использовать синтаксичскую разметку, а не хардкодить через биграммы, то есть искать существительные (NOUN) с зависимым именем собственным (PROPN)

2) Найти ключевые слова в корпусе текстов (лист 1). составить словарь именованных сущностей с помощью синтаксического парсера (лист 2) найти пересечение этих двух листов (лист словарь), расширить словарь с помощью эмбедингов

3) Найти ключевые слова для каждого товара (по всем комментариям, описывающим данный товар), найти их пересечение со словами, из названия продукта (лист упоминаний) Этот способ я буду реализовывать 

In [17]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [138]:
def split_text(a):
    b = a.copy()
    for s in b:
        if ' ' in s:
            b += s.split()
    return b

In [261]:
list_keys_names = {}

for name in data['product/title'].unique():
    df = data['review/text'][data['product/title'] == name].to_list()
    corpus = ' '.join(df)

    keys_summa = keywords.keywords(normalize_text(corpus), language='english', additional_stopwords=stop).split('\n')
    keys_textrank = kw(normalize_text(corpus), pos_filter=[], scores = False).split('\n')
    keys_rake = rake.run(normalize_text(corpus), minFrequency=2, maxWords=3)
    keys_rake = [x[0] for x in keys_rake]


    keys = keys_summa + keys_textrank + keys_rake
    names_tov = str(name).lower().split(' ')
    set_names = set(names_tov) & set(split_text(keys))
    list_keys_names[name] = list(set_names)



# list_keys_names

In [264]:
# list_keys_names

list_mentions - список упоминаний товаров в отзывах

In [265]:
list_mentions = []
for v in list_keys_names.values():
 list_mentions += v

collocations - список слов из list_mentions с правым и левым контекстом для каждого

texts - список со списками лемм каждого предложения (пригодится для коллокаций)

In [266]:
def normalize(text):
    # Take input text and return list of lemmas
    lemmas = []
    for t in tokenizer.tokenize(text):
        lemmas.append(m.parse(t)[0].normal_form)
    return lemmas

In [267]:
collocations = []
texts = []

list_not_null_mentions = {}
for k, v in list_keys_names.items():
  if v != []:
    list_not_null_mentions[k] = v

for key, value in list_not_null_mentions.items():
  little_data = ' '.join(data['review/text'][data['product/title'] == key])
  texts.append(normalize(little_data))
  for word in value:
    
      left_cont = re.findall(r'\b\w+\b\s' + word, little_data.lower())
      right_cont = re.findall(word + r'\s\b\w+\b', little_data.lower())
      collocations += left_cont + right_cont

set_collocations - множество коллокаций с упоминанием названия продукта

In [268]:
set_collocations = set(collocations)

In [269]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

In [270]:
bigram_measures = BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(texts)

функция взвращает пересечение set_collocations и списка всех коллокаций

In [272]:
def find_set(finder):
  itog = []
  for cort in finder:
    w2 = cort[0][0] + ' ' + cort[0][1]
    if w2 in set_collocations:
      itog.append(cort)
  return itog 

In [281]:
finder = finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [282]:
find_set(finder)[:20]

[(('to', 'play'), 3407.9514694145687),
 (('these', 'strings'), 2574.630290954189),
 (('set', 'up'), 2004.1761717667566),
 (('the', 'sound'), 1703.519655308398),
 (('long', 'time'), 1580.9723689628072),
 (('les', 'paul'), 1430.1748863249354),
 (('acoustic', 'guitar'), 1388.7524918884912),
 (('this', 'guitar'), 1200.2900212974591),
 (('gig', 'bag'), 1139.0055797762145),
 (('the', 'box'), 1129.4870482851147),
 (('music', 'store'), 1045.9807154120049),
 (('set', 'of'), 1005.3379774781506),
 (('the', 'neck'), 992.3495827144001),
 (('porcupine', 'tree'), 977.4998847653357),
 (('this', 'pedal'), 956.5572217306268),
 (('heavy', 'duty'), 954.0723979027337),
 (('the', 'first'), 947.4268789377857),
 (('to', 'learn'), 936.554496066292),
 (('phosphor', 'bronze'), 924.9780962361117),
 (('power', 'supply'), 911.8135568114358)]

In [283]:
finder = finder2.score_ngrams(bigram_measures.pmi)

In [284]:
find_set(finder)[:20]

[(('chili', 'peppers'), 19.77454900969144),
 (('kinte', 'kloth'), 19.77454900969144),
 (('squawky', 'shawm'), 19.77454900969144),
 (('ems', 'rosa'), 18.77454900969144),
 (('louis', 'vuitton'), 18.77454900969144),
 (('bluetake', 'bt450rx'), 18.189586508970287),
 (('lp634', 'cricket'), 18.189586508970287),
 (('rue', 'lepic'), 18.189586508970287),
 (('dangly', 'earrings'), 18.189586508970283),
 (('desgraciadamente', 'lm'), 18.189586508970283),
 (('dimarzio', 'x2n'), 18.189586508970283),
 (('micheal', 'kelly'), 18.189586508970283),
 (('onyx', '1220'), 18.189586508970283),
 (('pinkburst', 'playmate'), 18.189586508970283),
 (('rikki', 'rockett'), 18.189586508970283),
 (('20c', 'rocky'), 17.77454900969144),
 (('dunbar', 'polypencos'), 17.77454900969144),
 (('jp', 'boneyard'), 17.77454900969144),
 (('robelli', 'w4103fcs'), 17.77454900969144),
 (('vuitton', 'waltz'), 17.77454900969144)]

In [285]:
finder = finder2.score_ngrams(bigram_measures.dice)

In [286]:
itog = find_set(finder)
itog[:20]

[(('chili', 'peppers'), 1.0),
 (('kinte', 'kloth'), 1.0),
 (('squawky', 'shawm'), 1.0),
 (('vic', 'firth'), 0.95),
 (('hal', 'leonard'), 0.9230769230769231),
 (('les', 'paul'), 0.896969696969697),
 (('oscar', 'schmidt'), 0.8888888888888888),
 (('ridge', 'rider'), 0.8888888888888888),
 (('porcupine', 'tree'), 0.8521739130434782),
 (('ralph', 'hardimon'), 0.8333333333333334),
 (('austin', 'bazaar'), 0.8),
 (('bluetake', 'bt450rx'), 0.8),
 (('lp634', 'cricket'), 0.8),
 (('rue', 'lepic'), 0.8),
 (('zakk', 'wylde'), 0.782608695652174),
 (('seymour', 'duncan'), 0.7647058823529411),
 (('parasite', 'eve'), 0.7586206896551724),
 (('lee', 'oskar'), 0.75),
 (('tuna', 'melt'), 0.75),
 (('ems', 'rosa'), 0.6666666666666666)]

музыкальные инструменты часто называют именами известных музыкантов, в названии встречаются слова как-то характеризующие инструмент, например "детская черная гитара". Это усложняет поиск упоминаний продукта в комментариях.

таким образом, мера Дайса на мой взгляд лучшим образом ранжирует коллокации

In [259]:
def find_top(finder, list_words):
  itog = []
  for word in list_words:
    print('\n', word, '\n')
    for cort in finder:
      if (cort[0][0] == word) or (cort[0][1] == word):
        itog.append(cort)
        print(cort[0][0] + ' ' + cort[0][1])
  return itog 

выводит слова, относящиеся к ['pickup', 'hot', 'pedal', 'effects', 'danelectro']

In [260]:
list_words_to_show = list_mentions[:5]
list_top = find_top(finder, list_words_to_show)


 hot 

hot rod
hot tub
hot rails
hot shoe
hot head
virtual hot
hot car
digitech hot
blazing hot
hot chili
hot paf
hot potato
hot masters
ford hot
boiling hot
1b hot
deville hot
hot knife
aforementioned hot
hot swapping
hot resulting
hot summer
hot weather
hot leads
hot stack
hot wired
running hot
harps hot
hot axe
hot house
hot pin
hot glue
fender hot
run hot
hot water
strat hot
hot bridge
too hot
red hot
hot doing
hot digital
support hot
hot plug
hot room
hot days
big hot
hot quite
quite hot
very hot
hot even
never hot
hot at
hot buy
hot when
hot amp
little hot
hot also
hot there
so hot
the hot
your hot
get hot
as hot
a hot
hot or
or hot
one hot
hot so
my hot
hot that
hot and
are hot
hot as
s hot
is hot
on hot
hot in
hot i
and hot
hot this
hot to
to hot
hot it

 pickup 

bridge pickup
pickup pattern
neck pickup
piezo pickup
pickup selector
magnetic pickup
pickup warmer
rear pickup
jazz pickup
pickup alone
each pickup
buckled pickup
dirtyfinger pickup
pickup chrometo
realist pickup
ma